We built a model in 3.0-theberling-classifier.ipynb but we didn't have the right submissions. So now we're going to reverse engineer from their sample submission file to get the year, and team IDs that we need. Let's start by reading in their sample submission data file.

In [1]:
import pandas as pd

In [2]:
df_submission_sample = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv')

In [3]:
df_submission_sample.head()

,ID,Pred
0,2016_1112_1114,0.5
1,2016_1112_1122,0.5
2,2016_1112_1124,0.5
3,2016_1112_1138,0.5
4,2016_1112_1139,0.5


In [4]:
new = df_submission_sample["ID"].str.split("_", expand = True)
new

,0,1,2
0,2016,1112,1114
1,2016,1112,1122
2,2016,1112,1124
3,2016,1112,1138
4,2016,1112,1139
...,...,...,...
11385,2021,1452,1457
11386,2021,1452,1458
11387,2021,1455,1457
11388,2021,1455,1458


In [5]:
combinations = pd.DataFrame()
combinations['Season'] = new[0]
combinations['LowID'] = new[1]
combinations['HighID'] = new[2]
combinations['Season'] = pd.to_numeric(combinations['Season'])
combinations['LowID'] = pd.to_numeric(combinations['LowID'])
combinations['HighID'] = pd.to_numeric(combinations['HighID'])
combinations.head()

,Season,LowID,HighID
0,2016,1112,1114
1,2016,1112,1122
2,2016,1112,1124
3,2016,1112,1138
4,2016,1112,1139


Now let's load up the season averages data.

In [6]:
df = pd.read_csv('../input/2-compute-averages/data_averages.csv')

Let's merge the low ID team with it's data by season. To do that we'll temporarily rename LowID to TeamID and merge on Season and TeamID.

In [7]:
LowID = pd.merge(combinations.rename({'LowID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
LowID.head()

,Season,TeamID,HighID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2016,1112,1114,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
1,2016,1112,1122,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
2,2016,1112,1124,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
3,2016,1112,1138,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
4,2016,1112,1139,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0


In [8]:
LowID.columns = ['Season','LowID','HighID','LowScore','LowFGP','LowFGP3','LowFTP','LowOR','LowDR','LowAst','LowTO','LowStl','LowBlk','LowPF','LowRank','LowSeed']
LowID.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,LowTO,LowStl,LowBlk,LowPF,LowRank,LowSeed
0,2016,1112,1114,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
1,2016,1112,1122,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
2,2016,1112,1124,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
3,2016,1112,1138,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0
4,2016,1112,1139,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,12.818182,4.909091,4.787879,18.212121,16.870968,6.0


In [9]:
HighID = pd.merge(combinations.rename({'HighID': 'TeamID'}, axis=1), df, how="left", on=['Season','TeamID']).dropna().reset_index(drop=True)
HighID.head()

,Season,LowID,TeamID,Score,FGP,FGP3,FTP,OR,DR,Ast,TO,Stl,Blk,PF,Rank,Seed
0,2016,1112,1114,70.000000,0.458595,0.396232,0.727142,8.709677,24.193548,12.903226,10.225806,6.290323,3.064516,18.258065,55.457627,12.0
1,2016,1112,1122,74.727273,0.464163,0.352002,0.662456,11.090909,24.787879,12.878788,14.545455,6.909091,2.757576,17.636364,208.293103,16.0
2,2016,1112,1124,76.312500,0.464773,0.374205,0.721206,13.562500,23.218750,17.093750,12.812500,7.718750,3.968750,19.406250,24.229508,5.0
3,2016,1112,1138,76.606061,0.437268,0.332577,0.714150,12.090909,26.727273,12.545455,13.818182,6.727273,3.484848,21.272727,122.465517,14.0
4,2016,1112,1139,80.580645,0.466575,0.386546,0.726300,11.419355,25.000000,14.354839,10.225806,6.774194,3.354839,19.903226,34.241379,9.0


In [10]:
HighID.columns = ['Season','LowID','HighID','HighScore','HighFGP','HighFGP3','HighFTP','HighOR','HighDR','HighAst','HighTO','HighStl','HighBlk','HighPF','HighRank','HighSeed']
HighID.head()

,Season,LowID,HighID,HighScore,HighFGP,HighFGP3,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed
0,2016,1112,1114,70.000000,0.458595,0.396232,0.727142,8.709677,24.193548,12.903226,10.225806,6.290323,3.064516,18.258065,55.457627,12.0
1,2016,1112,1122,74.727273,0.464163,0.352002,0.662456,11.090909,24.787879,12.878788,14.545455,6.909091,2.757576,17.636364,208.293103,16.0
2,2016,1112,1124,76.312500,0.464773,0.374205,0.721206,13.562500,23.218750,17.093750,12.812500,7.718750,3.968750,19.406250,24.229508,5.0
3,2016,1112,1138,76.606061,0.437268,0.332577,0.714150,12.090909,26.727273,12.545455,13.818182,6.727273,3.484848,21.272727,122.465517,14.0
4,2016,1112,1139,80.580645,0.466575,0.386546,0.726300,11.419355,25.000000,14.354839,10.225806,6.774194,3.354839,19.903226,34.241379,9.0


And finally we can merge the HighID and LowID DataFrame on Season, LowID, HighID, and Win since those should be the same in each DataFrame.

In [11]:
model_dataset = pd.merge(LowID, HighID, how='outer', on=['Season','LowID','HighID'])
model_dataset.head()

,Season,LowID,HighID,LowScore,LowFGP,LowFGP3,LowFTP,LowOR,LowDR,LowAst,...,HighFTP,HighOR,HighDR,HighAst,HighTO,HighStl,HighBlk,HighPF,HighRank,HighSeed
0,2016,1112,1114,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,...,0.727142,8.709677,24.193548,12.903226,10.225806,6.290323,3.064516,18.258065,55.457627,12.0
1,2016,1112,1122,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,...,0.662456,11.090909,24.787879,12.878788,14.545455,6.909091,2.757576,17.636364,208.293103,16.0
2,2016,1112,1124,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,...,0.721206,13.562500,23.218750,17.093750,12.812500,7.718750,3.968750,19.406250,24.229508,5.0
3,2016,1112,1138,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,...,0.714150,12.090909,26.727273,12.545455,13.818182,6.727273,3.484848,21.272727,122.465517,14.0
4,2016,1112,1139,81.212121,0.48596,0.37945,0.725924,11.636364,28.606061,14.515152,...,0.726300,11.419355,25.000000,14.354839,10.225806,6.774194,3.354839,19.903226,34.241379,9.0


In [12]:
model_dataset.shape

(11390, 29)

In [13]:
model_dataset.to_csv('/kaggle/working/model_dataset2.csv',index=False)